# <center><b>  ISM 6562 Big Data for Business</b> </center>

### <center>  AIRLINE PERFORMANCE DATASET</center>

<center>by: Chukwuebuka Ezema  and Shruti Sridharan</center>
<br>
<br>
<br>

### <b>INTRODUCTION</b>
The U.S. Department of Transportation (DOT) Bureau of Transportation Statistics tracks the on-time performance of domestic flights. This data anlytics project aims to analyze the on-time flight departure performance and cancellations by major air carriers from Jan.2019 to Feb 2018
The data was downloaded from the U.S. Department of Transportation's (DOT) Bureau of Transportation Statistics (BTS):https://www.transtats.bts.gov/DL_SelectFields.asp <br>
<br>

#### <b>Exploratory Analysis:</b>
<b>Question 1</b><br>
Carrier with most flight cancellations.<br> 

<b>Question 2</b><br>
Consequently, which month and day of the week were flight cancellations most frequent?<br>

<b>Question 3</b><br>
What was the average delay time for flights origiating from Tampa (TPA)

<b>Question 4</b><br>
Which orgin city had the most delay?

<b>Research Question 5</b><br>
Origin city to Destination delay analysis

<b>Research Question 6</b><br>
Model


### <center>  EXPLORATORY ANALYSIS </center>

#### Carrier with most flight cancellations.<br> 

In [1]:
import findspark
findspark.init()
import pyspark

import pandas as pd
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql import SparkSession


sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = SQLContext(sc)
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
#Using panda dataframe to import the table
df = pd.read_csv(r'C:\Users\coe\Desktop\BigData\1.csv',sep=',')

In [3]:
#create a new copy of the panda dataframe
dfa= df

In [4]:
dfa.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_STATE_NM,DEST,DEST_STATE_NM,CRS_DEP_TIME,...,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,1,10,4,DL,3837,DTW,Michigan,CMH,Ohio,820,...,89.0,38.0,1.0,155.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10,4,DL,3838,IAD,Virginia,MSP,Minnesota,1658,...,170.0,148.0,1.0,908.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10,4,DL,3838,MSP,Minnesota,IAD,Virginia,1245,...,132.0,114.0,1.0,908.0,NaN,NaN,NaN,NaN,NaN,NaN
3,1,10,4,DL,3839,BHM,Alabama,LGA,New York,601,...,149.0,131.0,1.0,866.0,NaN,NaN,NaN,NaN,NaN,NaN
4,1,10,4,DL,3841,MSP,Minnesota,IAD,Virginia,2025,...,134.0,107.0,1.0,908.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
dfa.tail()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_STATE_NM,DEST,DEST_STATE_NM,CRS_DEP_TIME,...,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
638644,1,3,4,UA,4738,ORD,Illinois,LAN,Michigan,1535,...,60.0,34.0,1.0,179.0,NaN,NaN,NaN,NaN,NaN,NaN
638645,1,3,4,UA,4737,SGF,Missouri,ORD,Illinois,620,...,99.0,75.0,1.0,438.0,43.0,0.0,0.0,0.0,0.0,NaN
638646,1,3,4,UA,4736,ORD,Illinois,CAK,Ohio,745,...,85.0,56.0,1.0,343.0,NaN,NaN,NaN,NaN,NaN,NaN
638647,1,3,4,UA,4735,BIS,North Dakota,DEN,Colorado,615,...,102.0,76.0,1.0,516.0,NaN,NaN,NaN,NaN,NaN,NaN
638648,1,3,4,UA,4734,SDF,Kentucky,ORD,Illinois,955,...,99.0,53.0,1.0,286.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
dfa=dfa.loc[:, ~df.columns.str.contains('^Unnamed')]

In [7]:
#Create Schema to convert to Spark dataframe
schema = StructType([ StructField("Month", IntegerType(), True)\
                       ,StructField("Date", IntegerType(), True)\
                       ,StructField("Weekday", IntegerType(), True)\
                       ,StructField("Carrier", StringType(), True)\
                       ,StructField("FlightNo", IntegerType(), True)\
                       ,StructField("Origin", StringType(), True)\
                       ,StructField("OriginState", StringType(), True)\
                       ,StructField("Dest", StringType(), True)\
                       ,StructField("DestState", StringType(), True)\
                       ,StructField("SchDep", IntegerType(), True)\
                       ,StructField("DepDelay", FloatType(), True)\
                       ,StructField("DepDelay15", FloatType(), True)\
                       ,StructField("SchArr", IntegerType(), True)\
                       ,StructField("ArrDelay", FloatType(), True)\
                       ,StructField("Cancelled", FloatType(), True)\
                       ,StructField("Cancelcode", StringType(), True)\
                       ,StructField("Diverted", FloatType(), True)\
                       ,StructField("SchElapsedTime", FloatType(), True)\
                       ,StructField("ElapsedTime", FloatType(), True)\
                       ,StructField("Airtime", FloatType(), True)\
                       ,StructField("Flights", FloatType(), True)\
                       ,StructField("Distance", FloatType(), True)\
                       ,StructField("CarrierDelay", FloatType(), True)\
                       ,StructField("WeatherDelay", FloatType(), True)\
                       ,StructField("NASDelay", FloatType(), True)\
                       ,StructField("SecurityDelay", FloatType(), True)\
                       ,StructField("LateAircraftDelay", FloatType(), True)])

In [8]:
# initialize Spark session and create spark dataframe
spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("SparkSQL").getOrCreate()
dfr = spark.createDataFrame(dfa, schema=schema)
dfr.show(5)

+-----+----+-------+-------+--------+------+-----------+----+---------+------+--------+----------+------+--------+---------+----------+--------+--------------+-----------+-------+-------+--------+------------+------------+--------+-------------+-----------------+
|Month|Date|Weekday|Carrier|FlightNo|Origin|OriginState|Dest|DestState|SchDep|DepDelay|DepDelay15|SchArr|ArrDelay|Cancelled|Cancelcode|Diverted|SchElapsedTime|ElapsedTime|Airtime|Flights|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|
+-----+----+-------+-------+--------+------+-----------+----+---------+------+--------+----------+------+--------+---------+----------+--------+--------------+-----------+-------+-------+--------+------------+------------+--------+-------------+-----------------+
|    1|  10|      4|     DL|    3837|   DTW|   Michigan| CMH|     Ohio|   820|    -7.0|       0.0|   951|    -9.0|      0.0|       NaN|     0.0|          91.0|       89.0|   38.0|    1.0|   155.0|         NaN

In [9]:
dfr.createOrReplaceTempView("Airline")

In [10]:
#Total number of flights that took off in January
dfr1 = spark.sql("SELECT Carrier,Cancelled from Airline where Cancelled =0")

dfr1.count()

619099

In [11]:
#Total number of flights cancelled in January
dfr2 = spark.sql("SELECT Carrier,Cancelled from Airline where Cancelled =1")

dfr2.count()

19550

In [12]:
(dfr2.count()/dfr1.count())*100

3.1578148244464943

In [13]:
#Carrier with most cancellation
dfr2.groupby('Carrier').sum('Cancelled').orderBy('sum(Cancelled)', ascending=False).show()

+-------+--------------+
|Carrier|sum(Cancelled)|
+-------+--------------+
|     UA|        6114.0|
|     AA|        5832.0|
|     WN|        3949.0|
|     DL|        1513.0|
|     B6|         980.0|
|     AS|         736.0|
|     NK|         199.0|
|     F9|         151.0|
|     G4|          50.0|
|     HA|          26.0|
+-------+--------------+



In [14]:
#Weekday with most cancellations
dfr2 = spark.sql("SELECT Weekday, Cancelled from Airline where Cancelled =1")

t = {1:"Monday", 2: "Tuesday", 3:"Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}

conv = udf(lambda x: t[x], StringType())
dfwd= dfr2.withColumn("Day of Week", conv("Weekday")).groupBy('Day of Week').sum('Cancelled').orderBy('sum(Cancelled)', ascending=False).show()

+-----------+--------------+
|Day of Week|sum(Cancelled)|
+-----------+--------------+
|  Wednesday|        3959.0|
|    Tuesday|        3248.0|
|   Thursday|        3206.0|
|     Sunday|        2827.0|
|     Monday|        2814.0|
|   Saturday|        2481.0|
|     Friday|        1015.0|
+-----------+--------------+



In [15]:
#Average delay of flights originating from Tampa, JFK, ORD, LAX
dfrd = spark.sql("SELECT Origin, depDelay, Cancelled from Airline where (Origin='TPA' or Origin='ORD' or Origin='JFK' or Origin='LAX') and Cancelled=0")


In [16]:
dfrd.groupBy('Origin').avg('depDelay').orderBy('avg(depDelay)', ascending= False).show()

+------+------------------+
|Origin|     avg(depDelay)|
+------+------------------+
|   ORD|20.551402208660274|
|   JFK|  11.7518191520326|
|   LAX| 9.304769966142203|
|   TPA| 8.415590335815226|
+------+------------------+



In [17]:
# Origin city to Destination Delay
dfrod= spark.sql("SELECT concat(Origin,'-' ,Dest) as Route, depDelay, Cancelled from Airline where Cancelled=0")


In [18]:
dfrod.groupBy('Route').avg('depDelay').orderBy('avg(depDelay)', ascending= False).show(10)

+-------+------------------+
|  Route|     avg(depDelay)|
+-------+------------------+
|BWI-VPS|             237.0|
|VPS-BWI|             235.0|
|CMH-AUS|             218.0|
|AUS-CMH|             217.0|
|EWR-VPS|             208.5|
|TWF-SFO|199.14285714285714|
|RFD-PGD|             181.8|
|SLC-EGE|             163.0|
|EGE-SLC|             155.0|
|VPS-EWR|             144.5|
+-------+------------------+
only showing top 10 rows



In [19]:
# Origin city to Destination Delay
dfsd= spark.sql("SELECT concat(OriginState,'-' ,DestState) as Route, depDelay, Cancelled from Airline where Cancelled=0")


In [20]:
dfsd.groupBy('Route').avg('depDelay').orderBy('avg(depDelay)', ascending= False).show(10)

+--------------------+-----------------+
|               Route|    avg(depDelay)|
+--------------------+-----------------+
|Wyoming-South Dakota|            104.0|
|  Alabama-California|95.33333333333333|
|  Wyoming-New Jersey|79.41935483870968|
|    Wyoming-Illinois|68.81538461538462|
| Oklahoma-New Jersey|             66.0|
|North Dakota-Nort...|63.14473684210526|
|North Dakota-Georgia|             61.8|
|Puerto Rico-Kentucky|             58.5|
|  Montana-New Jersey|50.29032258064516|
|Mississippi-Illinois|49.11392405063291|
+--------------------+-----------------+
only showing top 10 rows



### <center><b>K-MEANS MODEL</b></center>

In [21]:
from scipy import stats
from sklearn.cluster import KMeans
import pandas as pd


df_tr = dfa
df_tr.head()

,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,MKT_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,ORIGIN_STATE_NM,DEST,DEST_STATE_NM,CRS_DEP_TIME,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,FLIGHTS,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,1,10,4,DL,3837,DTW,Michigan,CMH,Ohio,820,...,91.0,89.0,38.0,1.0,155.0,NaN,NaN,NaN,NaN,NaN
1,1,10,4,DL,3838,IAD,Virginia,MSP,Minnesota,1658,...,172.0,170.0,148.0,1.0,908.0,NaN,NaN,NaN,NaN,NaN
2,1,10,4,DL,3838,MSP,Minnesota,IAD,Virginia,1245,...,158.0,132.0,114.0,1.0,908.0,NaN,NaN,NaN,NaN,NaN
3,1,10,4,DL,3839,BHM,Alabama,LGA,New York,601,...,149.0,149.0,131.0,1.0,866.0,NaN,NaN,NaN,NaN,NaN
4,1,10,4,DL,3841,MSP,Minnesota,IAD,Virginia,2025,...,168.0,134.0,107.0,1.0,908.0,NaN,NaN,NaN,NaN,NaN


In [22]:
df_tr=df_tr.loc[df_tr['CANCELLED'] == 0]


In [23]:
#Standardize
clmns = ['DAY_OF_MONTH','DAY_OF_WEEK', 'DEP_DELAY', 'DEP_DEL15', 'DISTANCE']
df_tr_std = stats.zscore(df_tr[clmns])
print(df_tr_std)

[[-0.64297987  0.0838701  -0.34293458 -0.4621799  -1.05624232]
 [-0.64297987  0.0838701  -0.3227094  -0.4621799   0.23557658]
 [-0.64297987  0.0838701  -0.28225905 -0.4621799   0.23557658]
 ...
 [-1.42433679  0.0838701  -0.28225905 -0.4621799  -0.73371648]
 [-1.42433679  0.0838701  -0.28225905 -0.4621799  -0.4369241 ]
 [-1.42433679  0.0838701  -0.3227094  -0.4621799  -0.83150357]]


In [24]:
#Cluster the data
kmeans = KMeans(n_clusters=4, random_state=0).fit(df_tr_std)
labels = kmeans.labels_

#Glue back to originaal data
df_tr['clusters'] = labels

#Add the column into our list
clmns.extend(['clusters'])

#Lets analyze the clusters
print(df_tr[clmns].groupby(['clusters']).mean())

          DAY_OF_MONTH  DAY_OF_WEEK  DEP_DELAY  DEP_DEL15     DISTANCE
clusters                                                              
0            15.210701     3.836327  -0.590422   0.049031  1987.068080
1            13.880700     5.675505  -3.302270   0.000000   596.845417
2            17.154205     2.374383  -3.458916   0.000000   578.847151
3            16.399141     3.816378  73.046496   1.000000   724.930479


C:\Users\coe\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [26]:
# Load the data stored in csv format as a DataFrame.
data = spark.read.load(r'C:\Users\coe\Desktop\BigData\1.csv',
                     format="csv", sep=",", inferSchema='true', header="true")

In [27]:
data.printSchema()

root
 |-- MONTH: integer (nullable = true)
 |-- DAY_OF_MONTH: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- MKT_UNIQUE_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_STATE_NM: string (nullable = true)
 |-- DEST: string (nullable = true)
 |-- DEST_STATE_NM: string (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- DEP_DELAY: double (nullable = true)
 |-- DEP_DEL15: double (nullable = true)
 |-- CRS_ARR_TIME: integer (nullable = true)
 |-- ARR_DELAY: double (nullable = true)
 |-- CANCELLED: double (nullable = true)
 |-- CANCELLATION_CODE: string (nullable = true)
 |-- DIVERTED: double (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- ACTUAL_ELAPSED_TIME: double (nullable = true)
 |-- AIR_TIME: double (nullable = true)
 |-- FLIGHTS: double (nullable = true)
 |-- DISTANCE: double (nullable = true)
 |-- CARRIER_DELAY: double (nullable = tr

In [28]:
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col

stringIndexer = StringIndexer(inputCol="CRS_DEP_TIME", outputCol="index")
model = stringIndexer.fit(data)
df_data = model.transform(data)


In [707]:
df_data.rdd

MapPartitionsRDD[917] at javaToPython at NativeMethodAccessorImpl.java:0

In [29]:
#parse columns for classification(data) & Regression(datar)

data = df_data.rdd.map(lambda x: ((x[15]), Vectors.dense(float(x[4]),float(x[2]),float(x[9]),float(x[12]),float(x[21]))))
datar = df_data.rdd.map(lambda x: ((x[14]), Vectors.dense(float(x[4]),float(x[2]),float(x[9]),float(x[12]),float(x[21]))))
datalr = df_data.rdd.map(lambda x: ((x[14]), Vectors.dense(float(x[9]))))

In [31]:
from pyspark.ml.linalg import Vectors


In [32]:
# create dataframe for classification(datap) & Regression(datapr)

datap = sqlContext.createDataFrame(data, ['label','features'],samplingRatio=0.2)
datapr = sqlContext.createDataFrame(datar, ['label','features'],samplingRatio=0.2)

In [33]:
datlr = sqlContext.createDataFrame(datalr, ['label','features'],samplingRatio=0.2)

In [34]:
#Filter out null values

dfapp1=datap.filter("label is not null")
dfapp2=datapr.filter("label is not null")

In [35]:
dfapp3=datlr.filter("label is not null")

In [36]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel", handleInvalid="skip").fit(dfapp1)

In [37]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dfapp1)

In [38]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = dfapp1.randomSplit([0.7, 0.3])

In [39]:
# Train a DecisionTree model.
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [40]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

In [41]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [42]:
# Make predictions.
predictions = model.transform(testData)

In [43]:
# Select example rows to display.
predictions.select("prediction", "indexedLabel", "label").show(10)

+----------+------------+-----+
|prediction|indexedLabel|label|
+----------+------------+-----+
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
|       0.0|         2.0|    A|
+----------+------------+-----+
only showing top 10 rows



In [44]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.250691 


In [45]:
treeModel = model.stages[2]

In [46]:
# summary only
print(treeModel)

DecisionTreeClassificationModel (uid=DecisionTreeClassifier_803ab9947195) of depth 5 with 17 nodes


## RANDOM FOREST Classification

In [47]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString

# Train a RandomForest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

In [48]:
# Convert indexed labels back to original labels.
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                               labels=labelIndexer.labels)

In [49]:
# Chain indexers and forest in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

In [50]:
# Train model.  This also runs the indexers.
model = pipeline.fit(trainingData)

In [51]:
# Make predictions.
predictions = model.transform(testData)

In [52]:
# Select example rows to display.
predictions.select("predictedLabel", "label", "features").show(5)

+--------------+-----+--------------------+
|predictedLabel|label|            features|
+--------------+-----+--------------------+
|             B|    A|[43.0,2.0,1040.0,...|
|             B|    A|[166.0,3.0,2310.0...|
|             B|    A|[327.0,4.0,845.0,...|
|             B|    A|[415.0,5.0,2315.0...|
|             B|    A|[573.0,3.0,600.0,...|
+--------------+-----+--------------------+
only showing top 5 rows



In [53]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))

Test Error = 0.251728


In [54]:
rfModel = model.stages[2]
print(rfModel)  # summary only

RandomForestClassificationModel (uid=RandomForestClassifier_73fbad97bcb7) with 10 trees


## RANDOM FOREST Regression

In [55]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


In [56]:
# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4,handleInvalid='skip').fit(dfapp2)

In [57]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = dfapp2.randomSplit([0.7, 0.3])

In [58]:
# Train a RandomForest model.
rf = RandomForestRegressor(featuresCol="indexedFeatures")

In [59]:
# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

In [60]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [61]:
# Make predictions.
predictions = model.transform(testData)

In [62]:
# Select example rows to display.
predictions.select("prediction", "features", "label").show(5)

+--------------------+--------------------+-----+
|          prediction|            features|label|
+--------------------+--------------------+-----+
|0.008106605061505599|[1.0,2.0,800.0,11...|  0.0|
| 0.03234776669891556|[1.0,3.0,1140.0,1...|  0.0|
|0.008377390429297577|[1.0,4.0,800.0,11...|  0.0|
|0.005919031132843062|[1.0,5.0,800.0,11...|  0.0|
|0.017097053007050222|[1.0,6.0,800.0,11...|  0.0|
+--------------------+--------------------+-----+
only showing top 5 rows



In [63]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.171704


In [64]:
rfModel = model.stages[1]
print(rfModel)  # summary only

RandomForestRegressionModel (uid=RandomForestRegressor_ae343ca002f2) with 20 trees


## GRADIENT BOOSTED-TREE (Regression)

In [65]:
from pyspark.ml.regression import GBTRegressor

In [66]:
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(dfapp2)

In [67]:
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = dfapp2.randomSplit([0.7, 0.3])

In [68]:
# Train a GBT model.
gbt = GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

In [69]:
# Chain indexer and GBT in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, gbt])

In [70]:
# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

In [71]:
# Make predictions.
predictions = model.transform(testData)

In [72]:
# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

+--------------------+-----+--------------------+
|          prediction|label|            features|
+--------------------+-----+--------------------+
|0.004578731788893817|  0.0|[1.0,2.0,800.0,11...|
| 0.03330313775536682|  0.0|[1.0,3.0,1140.0,1...|
|0.001234406812453901|  0.0|[1.0,5.0,800.0,11...|
|0.012811186240871835|  0.0|[1.0,7.0,800.0,11...|
| 0.01004937241773574|  0.0|[2.0,1.0,1400.0,2...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [73]:
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 0.169945


In [74]:
gbtModel = model.stages[1]
print(gbtModel)  # summary only

GBTRegressionModel (uid=GBTRegressor_65b6bcf57e89) with 10 trees
